# Config

In [ ]:
import os

db_file = os.path.abspath("db.json")

# Get Structure from MP

In [ ]:
from pymatgen import Structure

struct_si = Structure.from_file("Si.cif")

# Firework 1: Structure Optimization

In [ ]:
from atomate.vasp.fireworks import OptimizeFW

fw1 = OptimizeFW(struct_si, vasp_cmd=None, db_file=db_file)

for task in fw1.tasks:
    print(task.fw_name)

# Firework 2: Static Calculation

In [ ]:
from atomate.vasp.fireworks import StaticFW

fw2 = StaticFW(struct_si, vasp_cmd=None, db_file=db_file, parents=[fw1])

for task in fw2.tasks:
    print(task.fw_name)

# Firework 3: Non-selfconsistent Calculation - 'uniform' mode

In [ ]:
from atomate.vasp.fireworks import NonSCFFW

fw3 = NonSCFFW(struct_si, vasp_cmd=None, db_file=db_file, 
               parents=[fw2], mode="uniform")

for task in fw3.tasks:
    print(task.fw_name)

# Firework 4: Non-selfconsistent Calculation - 'line' mode

In [ ]:
fw4 = NonSCFFW(struct_si, vasp_cmd=None, db_file=db_file, 
               parents=[fw2], mode="line")

for task in fw4.tasks:
    print(task.fw_name)

# Pack Fireworks into Workflow

In [ ]:
from fireworks import Workflow

fws = [fw1, fw2, fw3, fw4]

wf = Workflow(fws)

In [ ]:
%matplotlib inline
from fireworks.utilities.fw_utilities import plot_wf

plot_wf(wf)

# Emulate VASP

In [ ]:
from helper_functions import vasp_emulator

wf = vasp_emulator(wf)

# Add Workflow to Launchpad

In [ ]:
from fireworks import LaunchPad

lp = LaunchPad()
lp.reset("", require_password=False)

In [ ]:
lp.add_wf(wf)

# Run the workflow

In [ ]:
from fireworks import FWorker
from fireworks.core.rocket_launcher import rapidfire

rapidfire(lp)

# Retrieve results

In [ ]:
from atomate.vasp.database import VaspCalcDb

vasp_db = VaspCalcDb()

In [ ]:
from helper_functions import get_task_id

dos_task_id = get_task_id(lp, "Si-nscf uniform")
bs_task_id = get_task_id(lp, "Si-nscf line")
print(dos_task_id, bs_task_id)

In [ ]:
bs = vasp_db.get_band_structure(bs_task_id)
dos = vasp_db.get_dos(dos_task_id)